# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。





## requests + BeautifulSoup


In [26]:
import requests
from bs4 import BeautifulSoup as bs
import os
from PIL import Image
# url = 'https://www.google.com.tw/'
# r = requests.get(url)
# soup = bs(r.text)
# title = soup.find('title').text
# print(type(title),title)
# 速度較慢

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = bs(resp.text)
# 決定要儲存的資料夾
output_dir = 'downloads'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
image_tags = soup.find(id = 'main-content').findChildren('a', recursive=False)
image_tags
for img_tag in image_tags:
    # 取得所有圖片在第三方服務的 id
    if 'imgur' not in img_tag['href']:
        continue
    # 組合圖片而非網站的網址
    img_id = img_tag['href'].split('/')[-1]
    img_url = 'https://imgur.com/'+img_id+'.jpg'
    # 對圖片送出請求
    with requests.get(img_url,stream=True) as r:
        r.raise_for_status()
        img = Image.open(r.raw)
#         print(img.format.lower())
        img_savename = output_dir+'/'+img_id+'.'+img.format.lower()
#     print(img_savename)
        img.save(img_savename)
    print('Save image %s'%img_savename)


<class 'str'> Google
Save image downloads/Cgb5oo1.jpeg
Save image downloads/MgjHY4f.jpeg
Save image downloads/yeoRtAs.jpeg
Save image downloads/wsauNEW.jpeg
Save image downloads/IjiETcs.jpeg


## Grab + PyQuery

In [66]:
from grab import Grab
from pyquery import PyQuery as pq
import os
from PIL import Image
# g = Grab()
# r = g.go('https://www.google.com.tw/')
# doc = pq(r.body)
# title = doc('title')
# print(type(title.text()),title.text())
# 速度較快

g = Grab()
r = g.go('https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html',cookies={'over18': '1'})
doc = pq(r.body)
a = doc('#main-content')('a')
a = a.text()
a = a.split('  ')
imgs = [img+'.jpg' for img in a if 'imgur' in img]
for img in imgs:
    img_id = img.split('/')[-1]
    imgg = g.go(img)
    open(output_dir+'/'+img_id, 'wb').write(imgg.body)
    print(img_id)


Cgb5oo1.jpg
MgjHY4f.jpg
yeoRtAs.jpg
wsauNEW.jpg
IjiETcs.jpg
